# Analysis of students performance

## Set up block
*==> It must be executed minimum one time before executing next code blocks*

In [1]:
# === All imports ===
## global imports
import os
import sys
from itertools import groupby

## spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min, monotonically_increasing_id, round

# === Get data souce file ===
data_file = "./data/StudentsPerformance.csv"
if os.path.exists(data_file):
    print(f"[ok]: data source file is found: {data_file}")
else:
    print(f"[ko]: data source file is missing: {data_file}\n/!\The following code blocks will not be able to function normally until this problem is resolved")
    sys.exit(1)

# === Create the Spark session
try:
    ## avoid the error: java.io.IOException: Cannot run program "python3"
    os.environ["PYSPARK_PYTHON"] = "python"
    os.environ["PYSPARK_DRIVER_PYTHON"] = "python"
    ## create spark session
    spark = SparkSession.builder.master("local[*]").appName("# Analysis of students performance").config("spark.ui.showConsoleProgress", "true").getOrCreate()
    print("[ok]: spark session created")
except Exception as e:
    print(f"[ko]: cannot create spark session: {e}\n/!\The following code blocks will not be able to function normally until this problem is resolved")
    sys.exit(1)

[ok]: data source file is found: ./data/StudentsPerformance.csv
[ok]: spark session created


## Level 1: Basic DataFrame Manipulation

### Exercise 1.1 - Exploration
#### 1.1.1) Show the schema

In [2]:
df_student_performance = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .csv(data_file)
)
df_student_performance.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



#### 1.1.2) Count the number of students

In [3]:
nbr_of_students = df_student_performance.count()
print("nbr of students:", nbr_of_students)

nbr of students: 1000


#### 1.1.3) Display the first 10 lines

In [4]:
df_student_performance.show(10)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

#### 1.1.4) Display descriptive statistics (describe)

In [5]:
df_student_performance.describe().show()

+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  NULL|          NULL|                       NULL|        NULL|                   NULL|            66.089|            69.169|           68.054|
| stddev|  NULL|          NULL|                       NULL|        NULL|                   NULL|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          

### Exercise 1.2 - Selections and Filters
#### 1.2.1) Select only gender and the 3 scores

In [6]:
df_student_performance.select(
    "gender", "math score", "reading score", "writing score"
).show(10)

+------+----------+-------------+-------------+
|gender|math score|reading score|writing score|
+------+----------+-------------+-------------+
|female|        72|           72|           74|
|female|        69|           90|           88|
|female|        90|           95|           93|
|  male|        47|           57|           44|
|  male|        76|           78|           75|
|female|        71|           83|           78|
|female|        88|           95|           92|
|  male|        40|           43|           39|
|  male|        64|           64|           67|
|female|        38|           60|           50|
+------+----------+-------------+-------------+
only showing top 10 rows



#### 1.2.2) Filter students who obtained a grade above 90 in mathematics

In [7]:
df_student_performance.filter(col("math score") > 90).show(20)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group E|               some college|    standard|                   none|        97|           87|           82|
|  male|       group C|               some college|    standard|              completed|        98|           86|           90|
|female|       group E|          bachelor's degree|    standard|              completed|        99|          100|          100|
|  male|       group B|         associate's degree|    standard|              completed|        91|           89|           92|
|  male|       group E|         associate's degree|free/reduced|              completed|       100|     

#### 1.2.3) Filter students with lunch = "free/reduced"

In [8]:
df_student_performance.filter(col("lunch") == "free/reduced").show(20)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group B|               some college|free/reduced|                   none|        40|           43|           39|
|  male|       group D|                high school|free/reduced|              completed|        64|           64|           67|
|female|       group B|                high school|free/reduced|                   none|        38|           60|           50|
|female|       group B|           some high school|free/reduced|                   none|        18|     

#### 1.2.4) Count the number of students who have completed the preparatory course

In [9]:
nbr_of_students_with_completed_prep_course = df_student_performance.filter(col("test preparation course") == "completed").count()
print("nbr of students with completed prep course:", nbr_of_students_with_completed_prep_course)

nbr of students with completed prep course: 358


#### 1.2.5) Find the 10 best reading scores

In [10]:
df_student_performance.orderBy(col("reading score").desc()).limit(10).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group E|          bachelor's degree|    standard|                   none|       100|          100|          100|
|female|       group E|          bachelor's degree|    standard|              completed|        99|          100|          100|
|female|       group D|            master's degree|    standard|                   none|        87|          100|          100|
|  male|       group E|         associate's degree|free/reduced|              completed|       100|          100|           93|
|female|       group C|          bachelor's degree|    standard|              completed|        96|     

### Exercise 1.3 - Aggregations
#### 1.3.1) Calculate the average for each subject

In [11]:
df_student_performance.select(
    avg(col("math score")),
    avg(col("reading score")),
    avg(col("writing score")),
).show(20)

+---------------+------------------+------------------+
|avg(math score)|avg(reading score)|avg(writing score)|
+---------------+------------------+------------------+
|         66.089|            69.169|            68.054|
+---------------+------------------+------------------+



#### 1.3.2) Count the number of students by gender

In [12]:
df_student_performance.groupBy("gender").count().show()



+------+-----+
|gender|count|
+------+-----+
|female|  518|
|  male|  482|
+------+-----+



#### 1.3.3) Calculate the average score per gender

In [13]:
df_student_performance.groupBy("gender").agg(
    avg(col("math score")).alias("avg_math_score"),
    avg(col("reading score")).alias("avg_reading score"),
    avg(col("writing score")).alias("avg_writing_score")
).show()

+------+------------------+-----------------+-----------------+
|gender|    avg_math_score|avg_reading score|avg_writing_score|
+------+------------------+-----------------+-----------------+
|female|63.633204633204635|72.60810810810811|72.46718146718146|
|  male| 68.72821576763485|65.47302904564316|63.31120331950208|
+------+------------------+-----------------+-----------------+



#### 1.3.4) Find the maximum and minimum scores in math

In [14]:
df_student_performance.select(
    max(col("math score")).alias("max_math_score"),
    min(col("math score")).alias("min_math_score")
).show()

+--------------+--------------+
|max_math_score|min_math_score|
+--------------+--------------+
|           100|             0|
+--------------+--------------+



#### 1.3.5) Calculate the average per ethnic group (race/ethnicity)

In [15]:
df_student_performance.groupBy("race/ethnicity").agg(
    avg(col("math score")).alias("avg_math_score"),
    avg(col("reading score")).alias("avg_reading_score"),
    avg(col("writing score")).alias("avg_writing_score")
).show()

+--------------+-----------------+-----------------+------------------+
|race/ethnicity|   avg_math_score|avg_reading_score| avg_writing_score|
+--------------+-----------------+-----------------+------------------+
|       group B|63.45263157894737|67.35263157894737|              65.6|
|       group C|64.46394984326018|69.10344827586206| 67.82758620689656|
|       group D|67.36259541984732|70.03053435114504| 70.14503816793894|
|       group A|61.62921348314607|64.67415730337079|62.674157303370784|
|       group E|73.82142857142857|73.02857142857142| 71.40714285714286|
+--------------+-----------------+-----------------+------------------+



## Level 2 : Joints

### Create a table of rating categories

In [16]:
df_grades_ref = spark.createDataFrame([
    ("A", 90, 100),
    ("B", 80, 89),
    ("C", 70, 79),
    ("D", 60, 69),
    ("F", 0, 59)
], ["grade", "min_score", "max_score"])

df_grades_ref.show()

+-----+---------+---------+
|grade|min_score|max_score|
+-----+---------+---------+
|    A|       90|      100|
|    B|       80|       89|
|    C|       70|       79|
|    D|       60|       69|
|    F|        0|       59|
+-----+---------+---------+



### Create a table of departments

In [17]:
df_departments = spark.createDataFrame([
    ("group A", "Sciences"),
    ("group B", "Arts"),
    ("group C", "Commerce"),
    ("group D", "Ingénierie"),
    ("group E", "Médecine")
], ["ethnicity", "department"])

df_departments.show()

+---------+----------+
|ethnicity|department|
+---------+----------+
|  group A|  Sciences|
|  group B|      Arts|
|  group C|  Commerce|
|  group D|Ingénierie|
|  group E|  Médecine|
+---------+----------+



### Exercise 2.1 - Simple Join

#### 2.1.1) Join students with departments in the race/ethnicity column. Display: gender, ethnicity, department, math score

In [18]:
df_inner_std_dep = df_student_performance.join(df_departments, df_student_performance["race/ethnicity"] == df_departments["ethnicity"], how="inner").select(
    col("gender"),
    col("ethnicity"),
    col("department"),
    col("math score"),
    col("writing score"),
    col("reading score")
)
df_inner_std_dep.show(20)

+------+---------+----------+----------+-------------+-------------+
|gender|ethnicity|department|math score|writing score|reading score|
+------+---------+----------+----------+-------------+-------------+
|  male|  group A|  Sciences|        63|           62|           63|
|female|  group A|  Sciences|        44|           45|           45|
|  male|  group A|  Sciences|        57|           54|           51|
|female|  group A|  Sciences|        78|           91|           87|
|female|  group A|  Sciences|        54|           67|           63|
|female|  group A|  Sciences|        53|           60|           50|
|  male|  group A|  Sciences|        66|           64|           68|
|female|  group A|  Sciences|        56|           64|           58|
|  male|  group A|  Sciences|        61|           61|           62|
|  male|  group A|  Sciences|        67|           53|           57|
|female|  group A|  Sciences|        69|           82|           84|
|female|  group A|  Sciences|     

#### 2.1.2) Count the number of students per department

In [19]:
df_inner_std_dep.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|  Sciences|   89|
|      Arts|  190|
|  Commerce|  319|
|Ingénierie|  262|
|  Médecine|  140|
+----------+-----+



#### 2.1.3) Calculate the average scores per department

In [20]:
df_inner_std_dep.groupBy("department").agg(
    avg(col("math score")).alias("avg_math_score"),
    avg(col("reading score")).alias("avg_reading_score"),
    avg(col("writing score")).alias("avg_writing_score")
).show()

+----------+-----------------+-----------------+------------------+
|department|   avg_math_score|avg_reading_score| avg_writing_score|
+----------+-----------------+-----------------+------------------+
|  Sciences|61.62921348314607|64.67415730337079|62.674157303370784|
|      Arts|63.45263157894737|67.35263157894737|              65.6|
|  Commerce|64.46394984326018|69.10344827586206| 67.82758620689656|
|Ingénierie|67.36259541984732|70.03053435114504| 70.14503816793894|
|  Médecine|73.82142857142857|73.02857142857142| 71.40714285714286|
+----------+-----------------+-----------------+------------------+



### Exercise 2.2 - Transformation and Join

#### 2.2.1) Create a DataFrame with student_id and average score &
#### Calculate the average of the 3 subjects for each student

In [21]:
df_students_with_id = df_student_performance.withColumn("student_id", monotonically_increasing_id())
df_students_with_id.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|student_id|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|         0|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|         1|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|         2|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|         3|
|  male|       group C|    

In [22]:
df_students_with_id_with_avg_score = df_students_with_id.withColumn(
    "avg_score",
    round(
        (col("math score") + col("reading score") + col("writing score")) / 3,
        2
    )
)
df_students_with_id_with_avg_score.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+---------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|student_id|avg_score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+---------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|         0|    72.67|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|         1|    82.33|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|         2|    92.67|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|       

#### 2.2.2) "Join" this DataFrame with the df_grades_ref table
#### To assign a grade to each student
#### Tip: Use a join condition with BETWEEN
#### Example: (avg_score >= min_score) & (avg_score <= max_score)

In [23]:
df_inner_students_with_grades_ref = df_students_with_id_with_avg_score.join(df_grades_ref, df_students_with_id_with_avg_score["avg_score"].between(df_grades_ref["min_score"], df_grades_ref["max_score"]))
df_inner_students_with_grades_ref.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+---------+-----+---------+---------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|student_id|avg_score|grade|min_score|max_score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+---------+-----+---------+---------+
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|         2|    92.67|    A|       90|      100|
|female|       group B|               some college|    standard|              completed|        88|           95|           92|         6|    91.67|    A|       90|      100|
|  male|       group C|               some college|    standard|              completed|        98|           86|           9

#### 2.2.3) Count how many students have each grade (A, B, C, D, F)

In [24]:
df_inner_students_with_grades_ref.groupBy("grade").count().show()

+-----+-----+
|grade|count|
+-----+-----+
|    A|   52|
|    B|  140|
|    C|  245|
|    D|  241|
|    F|  274|
+-----+-----+



### Exercise 2.3 - Cross Analysis
#### 2.3.1) Join students with departments &
#### Calculate the average by department AND by gender

In [25]:
df_student_dep = df_student_performance.join(df_departments, df_student_performance["race/ethnicity"] == df_departments["ethnicity"], how="left")
df_student_dep.show(10)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+---------+----------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|ethnicity|department|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+---------+----------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|  group B|      Arts|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|  group B|      Arts|
|female|       group B|         associate's degree|    standard|                   none|        71|           83|           78|  group B|      Arts|
|female|       group B|               some college|    standard|              completed|        88|       

In [26]:
df_student_dep_with_avg_score = df_student_dep.withColumn("avg_score", round(
     (col("math score") + col("reading score") + col("writing score")) / 3,
        2
))
df_student_dep_with_avg_score.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+---------+----------+---------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|ethnicity|department|avg_score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+---------+----------+---------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|  group B|      Arts|    72.67|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|  group B|      Arts|    92.67|
|female|       group B|         associate's degree|    standard|                   none|        71|           83|           78|  group B|      Arts|    77.33|
|female|       group B|               some col

##### The average by department

In [27]:
df = df_student_dep_with_avg_score.groupBy("department").agg(
    avg(col("avg_score")).alias("avg_score"),
)
df.show()

+----------+-----------------+
|department|        avg_score|
+----------+-----------------+
|  Commerce|67.13156739811913|
|  Sciences|62.99213483146067|
|  Médecine|72.75200000000001|
|Ingénierie|69.17938931297712|
|      Arts|65.46857894736843|
+----------+-----------------+



##### The average by gender

In [28]:
df = df_student_dep_with_avg_score.groupBy("gender").agg(
    avg(col("avg_score")).alias("avg_score"),
)
df.show()

+------+-----------------+
|gender|        avg_score|
+------+-----------------+
|female|69.56949806949811|
|  male|65.83730290456432|
+------+-----------------+



#### 2.3.2) Identify the department with the best results

In [31]:
dep_with_best_results = df_student_dep_with_avg_score.orderBy(col("avg_score").desc()).limit(1).select("department").show()

+----------+
|department|
+----------+
|  Médecine|
+----------+

